In [ ]:
#This code needs a high memory machine to run

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import pymn
import anndata as ad
import time
import os
from pyprojroot import here
import resource

In [2]:
base_data_folder = "/vault/lfrench/mouse_brain_cluster_replicability/data/"

In [3]:
#from braincelldata.org
adata_macosko = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/macosko/from_google_drive/Macosko_Mouse_Atlas_Single_Nuclei.Use_Backed.h5ad")

In [4]:
adata_macosko.obs['cell.type'] = adata_macosko.obs['ClusterNm'].astype(str)
adata_macosko.obs['study_id'] = "Macosko"
adata_macosko.obs.index = adata_macosko.obs.index.to_numpy(dtype="str")

In [5]:
print("Number of Macosko cell types: " + str(len(set(adata_macosko.obs["cell.type"]))))


Number of Macosko cell types: 5030


In [ ]:
##########
### Zeng
##########

In [6]:
adata_Zeng = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome.h5ad")

In [7]:
adata_Zeng.obs.groupby('method').count()

/tmp/ipykernel_1260206/781402409.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adata_Zeng.obs.groupby('method').count()


,library_prep,gene.counts.0,doublet_score,roi,umi.counts,sex,external_donor_name,age,medical_conditions,cl
method,,,,,,,,,,
10Xv2,1700767,1700767,1700767,1700767,1700767,1700767,1700767,1700767,1700767,1700767
10Xv3,2342082,2342082,2342082,2342082,2342082,2342082,2342082,2342082,2342082,2342082
10xMulti,1687,1687,1687,1687,1687,1687,1687,1687,1687,1687


In [8]:
base_data_folder + "whole_mouse_brain/zeng/from_aws/"

'/vault/lfrench/mouse_brain_cluster_replicability/data/whole_mouse_brain/zeng/from_aws/'

In [9]:
Zeng_cluster_info_AWS = pd.read_csv(base_data_folder + "/whole_mouse_brain/zeng/from_aws/AIT21.0/AIT21_annotation_freeze_081523.tsv", sep = '\t')

In [10]:
#set study ID here to prevent view as actual memory grabs 
Zeng_cluster_info_AWS['study_id'] = "Zeng"
Zeng_cluster_info_AWS['cell.type'] = Zeng_cluster_info_AWS['cl'].astype(str)

In [11]:
#set it to be strings
adata_Zeng.obs['cl'] = adata_Zeng.obs['cl'].astype(str)
Zeng_cluster_info_AWS['cl'] = Zeng_cluster_info_AWS['cl'].astype(str)

In [12]:
adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)
adata_Zeng.obs = adata_Zeng.obs.merge(Zeng_cluster_info_AWS, how="left", on='cl')

In [13]:
#use ensmbl ID as index
adata_Zeng.var = adata_Zeng.var.set_index("gene_identifier")

In [14]:
#cells lost
adata_Zeng.obs['cluster_id'].isna().sum() #none it seems

0

In [15]:
#LQ or low quality cells
sum(adata_Zeng.obs['subclass_label'] == "LQ")

1560

In [16]:
#remove #LQ 
adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)
adata_Zeng.obs["filter_out"] = adata_Zeng.obs['subclass_label'] == "LQ"
adata_Zeng = adata_Zeng[~adata_Zeng.obs["filter_out"],:]
adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)

/tmp/ipykernel_1260206/299110530.py:5: ImplicitModificationWarning: Trying to modify index of attribute `.obs` of view, initializing view as actual.
  adata_Zeng.obs.index = adata_Zeng.obs.index.astype(str)


In [17]:
print("Number of Zeng cell types: " + str(len(set(adata_Zeng.obs["cell.type"]))))

Number of Zeng cell types: 5322


In [18]:
seed = 0  # change for different random splits
rng = np.random.default_rng(seed)

n_mac = adata_macosko.n_obs
n_zeng = adata_Zeng.n_obs
n_part = min(n_mac, n_zeng) // 2

In [21]:
n_part

2021488

In [23]:
perm_mac = rng.permutation(n_mac)[: 2 * n_part]
macA_idx = perm_mac[:n_part]
macB_idx = perm_mac[n_part: 2 * n_part]

In [24]:
adata_macosko.obs

,ClusterKey,ClusterNm,derived_cell_libs,cell.type,study_id
pBICCNsMMrBSL1aiM007d190529_CAGTTCCTCTATCGGA,300-0-0-0-0-0-0-0-11-0-0-1-0-0-1-0,Ex_Lhx2_Col5a2_1,pBICCNsMMrBSL1aiM007d190529,Ex_Lhx2_Col5a2_1,Macosko
pBICCNsMMrBSL1aiM007d190529_GATCAGTCACCCTGTT,300-0-0-0-0-0-0-0-0-0-0-0-1-1-0-0-0-0-0-0-1-1,Inh_Dmrt3_Prdm6_2,pBICCNsMMrBSL1aiM007d190529,Inh_Dmrt3_Prdm6_2,Macosko
pBICCNsMMrBSL1aiM007d190529_CTGGCAGAGGGAACAA,300-0-0-0-0-0-0-4-1-0-0-0-0-1-0-0-0-0-0-0,Inh_Pax2_Plscr2_1,pBICCNsMMrBSL1aiM007d190529,Inh_Pax2_Plscr2_1,Macosko
pBICCNsMMrBSL1aiM007d190529_GTAATGCAGCGCATCC,300-0-0-0-0-0-0-0-0-0-14-1-0-1-1,Inh_Tfap2b_Slc22a7,pBICCNsMMrBSL1aiM007d190529,Inh_Tfap2b_Slc22a7,Macosko
pBICCNsMMrBSL1aiM007d190529_ACTTCCGGTGGTCCCA,300-0-0-0-6-0-1-0-0-0,Chol_Isl2_Gata6_1,pBICCNsMMrBSL1aiM007d190529,Chol_Isl2_Gata6_1,Macosko
...,...,...,...,...,...
pBICCNsMMrHPFCA2CA3iF014d210217A2_CCCTGATAGCGAGTAC,301-1-0,Micro_Selplg_Siglech,pBICCNsMMrHPFCA2CA3iF014d210217A2,Micro_Selplg_Siglech,Macosko
pBICCNsMMrHPFCA2CA3iF014d210217A2_AGCGTCGTCAGACTGT,300-1-1-0-1-0-0,Astro_Ephb1_Fzd2,pBICCNsMMrHPFCA2CA3iF014d210217A2,Astro_Ephb1_Fzd2,Macosko
pBICCNsMMrHPFCA2CA3iF014d210217A2_AGCGTCGGTGAGGATC,300-2-0-0,Oligo_Opalin_Synpr,pBICCNsMMrHPFCA2CA3iF014d210217A2,Oligo_Opalin_Synpr,Macosko
pBICCNsMMrHPFCA2CA3iF014d210217A2_TGCAGGCCACAAGCTT,300-1-1-0-1-0-0,Astro_Ephb1_Fzd2,pBICCNsMMrHPFCA2CA3iF014d210217A2,Astro_Ephb1_Fzd2,Macosko


In [25]:
base_data_folder + "/whole_mouse_brain/macosko/macosko_A.h5ad"

'/vault/lfrench/mouse_brain_cluster_replicability/data//whole_mouse_brain/macosko/macosko_A.h5ad'

In [28]:
base_data_folder + "/whole_mouse_brain/processed/macosko/subsets/Macosko_Mouse_Atlas_Single_Nuclei.A."+str(n_part)+".h5ad"

'/vault/lfrench/mouse_brain_cluster_replicability/data//whole_mouse_brain/processed/macosko/subsets/Macosko_Mouse_Atlas_Single_Nuclei.A.2021488.h5ad'

In [31]:
adata_macosko[macA_idx].write_h5ad(base_data_folder + "/whole_mouse_brain/processed/macosko/subsets/Macosko_Mouse_Atlas_Single_Nuclei.A."+str(n_part)+".h5ad")
print("Done writing A")

/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/anndata/_core/anndata.py:1138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/anndata/_core/anndata.py:1138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


Done writing A


In [32]:
adata_macosko[macB_idx].write_h5ad(base_data_folder + "/whole_mouse_brain/processed/macosko/subsets/Macosko_Mouse_Atlas_Single_Nuclei.B."+str(n_part)+".h5ad")
print("Done writing B")

/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/anndata/_core/anndata.py:1138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/anndata/_core/anndata.py:1138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


Done writing B


In [34]:
#free up space
del adata_macosko

In [35]:
perm_z = rng.permutation(n_zeng)[: 2 * n_part]
zA_idx = perm_z[:n_part]
zB_idx = perm_z[n_part: 2 * n_part]

In [36]:
adata_Zeng[zA_idx].write_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome.A."+str(n_part)+".h5ad")
print("Done writing A")

/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/anndata/_core/anndata.py:1138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/anndata/_core/anndata.py:1138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/anndata/_core/anndata.py:1138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/anndata/_core/anndata.py:1138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/anndata/_core/anndata.py:1138: ImplicitModific

Done writing A


In [37]:
adata_Zeng[zB_idx].write_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome.B."+str(n_part)+".h5ad")
print("Done writing B")

/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/anndata/_core/anndata.py:1138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/anndata/_core/anndata.py:1138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/anndata/_core/anndata.py:1138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/anndata/_core/anndata.py:1138: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/home/leon/miniconda3/envs/mouse_brain_cells/lib/python3.10/site-packages/anndata/_core/anndata.py:1138: ImplicitModific

Done writing B
